In [6]:
%%time
#print("1")
import tensorflow as tf
from numba import cuda
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras.preprocessing.sequence import pad_sequences
#print("2")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import pickle
from keras.layers import Dense, Input, Dropout
#print("3")
from keras import Sequential
#print("4")
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import keras
print(keras.__version__)
from sklearn.model_selection import train_test_split
from keras.constraints import maxnorm
import numpy as np

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1331614535948791056
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7473294746
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17851818086571483571
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
2.3.1
Wall time: 1.1 s


In [7]:
%%time

with open(r"comments.txt", "rb") as f:
    clean_train_comments = pickle.load(f) 
    f.close()

with open(r"targets.txt", "rb") as ft:
    y= pickle.load(ft) 
    ft.close()

    
y = [int(s) for s in y]



#tfidf vectorization
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

# We transform each complaint into a vector
print("training tfidf and tranforming")
X = tfidf.fit_transform(clean_train_comments).toarray() #clean-train_comments # as this: https://stats.stackexchange.com/questions/154660/tfidfvectorizer-should-it-be-used-on-train-only-or-traintest   and this: https://stackoverflow.com/questions/47778403/computing-tf-idf-on-the-whole-dataset-or-only-on-training-data suggest,train tfidf only on training set
vocab_size = len(tfidf.vocabulary_) + 1
print("vocab_size = ", vocab_size)
# evaluate max len train data
maxlen = max([len(x) for x in X])
# pad train data accordingly
print("padding")
X_pad =  pad_sequences(X, padding='post', maxlen=maxlen, dtype='float32') 

print("done")

training tfidf and tranforming
vocab_size =  120536
padding
done
Wall time: 1min 50s


In [8]:
# Define the models.

def model0():  # from https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

    METRICS = [ 
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.AUC(name='auc'),
    ]

    optimizer="Adamax" #"adam"
    dropout=0.1 #0.1
    init='uniform'
    nbr_features= vocab_size-1 #2500
    dense_nparams=256

    model = Sequential()
    model.add(Dense(dense_nparams, activation='softsign', input_shape=(nbr_features,), kernel_initializer=init,  kernel_constraint=maxnorm(3))) # maxnorm(0) & softmax & sigmoid -> 0.89 # maxnorm(0) & softmax & softmax -> 0.5  maxnorm(2) & relu & sigmoid ->0.92  maxnorm(1) & relu & sigmoid ->0.82
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid')) # relu & "softmax" fa 0.5-> non va bene  #' relu & softplus' -> 0.75 #'sigmoid'
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = METRICS)
    return model
    

In [9]:

model = model0()

history = model.fit(x=X_pad, y=y, batch_size = 8, epochs = 3, verbose=10, shuffle=True, max_queue_size=10, workers=4, use_multiprocessing=True) #, callbacks=callbacks , validation_split=0.2

# reset gpu memory https://stackoverflow.com/a/60354785/13110508 (but be warned: it crashes python, so use it just at the end)
# device = cuda.get_current_device()
# device.reset()

Epoch 1/3
Epoch 2/3
Epoch 3/3


In [10]:
import pandas as pd
with open(r"comments_test.txt", "rb") as f:
    clean_test_comments = pickle.load(f) 
    f.close()
    
    
X_test = tfidf.transform(clean_test_comments).toarray()
maxlen_test = max([len(x) for x in X_test])
X_test_pad = pad_sequences(X_test, padding='post', maxlen=maxlen, dtype='float32')
#X_test_pad_scal = scaler.transform(X_test_pad)

y_pred = model.predict_proba(X_test_pad)
y_pred_unp = [y_pred[i][0] for i in range(len(y_pred))]
with open(r"authors_test.txt", "rb") as f:
    authors = pickle.load(f) 
    f.close()
    
solution = pd.DataFrame({"author":authors, "gender":y_pred_unp})

solution.to_csv(r"Q:\tooBigToDrive\data-mining\kaggle\data\challengedadata\solutions\simpleNetNoScalProbaGridD0_sol.csv",index = False)